[Tensorflow Object Detection Tuturial](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html#)

# Custom (Image) Data 구하기

# Custom (Image) Data Labeling

# 전단계
- 구글드라이브 연결
- 상대경로로 할 것이므로 Process.ipynb 있는 디렉토리로 이동
- workspace/images 에 이미지 데이터셋 넣고 압축 푼다. 

# Tensorflow Object Detection 2 API 설치
1. clone 
    - `!git clone https://github.com/tensorflow/models.git`
1. PYTHONPATH 환경설정에 models/research 추가  
1. 필요 모듈 설치
    - `!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk`
    - `!pip install -qq Cython contextlib2 pillow lxml matplotlib pycocotools`
1. proto 파일 컴파일
    - models/research 경로로 이동
        - `%cd models/research`
    - `protoc .object_detection/protos/*.proto --python_out=.`
1. setup.py 를 이용해 필요한 모듈 추가 설치
    - setup.py를 현재 디렉토리로 카피
        - `!cp object_detection/packages/tf2/setup.py . `
    - 설치
        - `!python -m pip install . `
    - 설치 확인 - 아래 스크립트 실행시 오류 없이 실행되면 설치 잘 된 것임.
        - `!python object_detection/builders/model_builder_tf2_test.py`
1. 원래 디렉토리로 이동
    - `%cd ../..`        

# 경로 설정

# Label Map 생성

# TFRecord 생성

# Pretrained Model Download
- tf2 detection Model Zoo: https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md
- SSD MobileNet V2 FPNLite 320x320 다운로드
    - 성능은 떨어지지만 학습속도가 빠르다.

# Model pipeline.config 파일 카피 및 설정 변경

## Model pipeline.config 파일 카피
- pretrained 모델의 압축을 풀면 pipeline.config 파일이 있다.
- pipeline.config 설정파일
    - 모델관련 설정
    - Training 관련 설정
- 이 파일을 workspace\model 로 copy 한다.

## pipeline.config 설정 변경
- pipeline.config 내용 변경은 파일을 **직접 변경**할 수도 있고 **코드상에서 변경**할 수도 있다.

### 필수 변경사항
-  class개수 변경
-  train 배치 사이즈 변경 - gpu 메모리 사양에 맞게 변경한다.
-  pretrained model 경로 설정
-  pretrained model이 어떤 종류의 모델인지 설정
-  train 관련 변경
    -  labelmap 파일 경로 설정
    -  train 용 tfrecord 파일 경로 지정
-  evaluation 관련 변경
    -  labelmap 파일 경로 설정
    -  evaluation 용 tfrecord 파일 경로 지정

# custom data를 이용한 Model 학습
- 다음 명령어를 실행한다.
- 시간이 오래 걸리므로 terminal에서 실행한다.
```
python models/research/object_detection/model_main_tf2.py --model_dir=workspace/model/checkpoint --pipeline_config_path=workspace/model/pipeline.config --num_train_steps=3000
```

## 옵션
- model_dir: 학습한 모델의 checkpoint 파일을 저장할 경로. (1000 step당 저장한다.)
- pipeline_config_path: pipeline.config 파일 경로
- num_train_steps: 학습할 step 수

# 학습한 모델 추출(export)
- `models/research/object_detection/exporter_main_v2.py` 사용
- 옵션
    - `exporter_main_v2.py --helpshort || exporter_main_v2.py --helpfull`
    - input_type : input node type
        - image_tensor, encoded_image_string_tensor
    - train_checkpoint: 학습된 checkpoint 파일이 저장된 경로(folder/directory)
    - pipeline_config_path: pipeline.config 파일의 경로 (파일명 포함)
    - output_directory: export된 모델을 저장할 경로.
- 추출된 디렉토리 구조
```bash
output_dir
├─ checkpoint/
├─ save_model/
└─ pipeline.config
```
    - checkpoint: custom data 학습한 checkpoint 파일들을 이 디렉토리로 복사한다.
    - save_model: pipeline.config 설정에 맞춰 생성된 model
    - pipeline.config: pipeline.config 설정파일

# Inference

### 사용 함수,메소드
-  ### tf.convert_to_tensor(array_like, dtype)
    - array_like 를 Tensoflow Tensor 객체로 변환
    - `tf.convert_to_tensor([[1,2],[3,4]])`
- ### detection_model.preprocess(image 4차원 ndarray)
    - 전달받은 이미지를 model의 input shape에 맞게 resizing 한다.
    - 반환값: (resize된 image Tensor, 이미지의 shape) 을 tuple로 반환
- ### detection_model.predict(image tensor, shape tensor)
    - 추론/detection 메소드
    - 이미지, image shape을 받아서 detection한 결과를 딕셔너리로 반환한다.
    - 반환 dictionary key
        - **preprocessed_inputs**:  입력 이미지 Tensor. preprocess()로 처리된 이미지. 
        - **feature_maps**: List. feature map 들을 반환
        - **anchors**: 2D Tensor. normalize 된 anchor box들의 좌표를 반환. 2-D float tensor: \[num_anchors, 4\]
        - **final_anchors**: 3D Tensor. batch 당 anchors. (anchors에 batch가 포함된 것). \[batch_size, num_anchors, 4\]
        - **box_encodings**: 3D flost tensor. predict한 box들의 normalize된 좌표. \[batch_size, num_anchors,box_code_dimension\]
        - **class_predictions_with_background**: 3D Tensor. 클래스 확률을 반환.(logit). \[batch_size, num_anchors, num_classes+1]\
            - background 확률을 포함해서 num_classes+1개가 된다. (index 0: background)
            
- ### detection_model.postprocess(prediction_dict, shape)
    - predict()가 예측한 결과 딕셔너리를 받아서 최종 Detection 결과를 반환한다.
    - predict()는 anchor별로 예측결과를 모아서 주고 postprocess는 최종 결과를 추출해서 반환.
    - 반환 dictionary key
        - **num_detections**: Detect한 개수 (bounding box 개수)
        - **detection_boxes**: [batch, max_detections, 4]. 후처리한 detection box
        - **detection_scores**: [batch, max_detections]. post-processed detection box들의 scalar scores.
        - **detection_classes**: [batch, max_detections] tensor with classes for post-processed detection classes.
        - **raw_detection_boxes**:[batch, total_detections, 4] Non-Max Suppression 하기 전의 감지된 box들
        - **raw_detection_scores**: [batch, total_detections, num_classes_with_background]. raw detection box들의 class별 점수
        - **detection_multiclass_scores**: [batch, max_detections, num_classes_with_background] tensor with class score distribution for post-processed detection boxes including background class if any.
        - **detection_anchor_indices**: [batch, max_detections] tensor with classes for post-processed detection classes.